In [2]:
'''Methods Validation'''

import sys, os, pathlib

# directory corrente del notebook
NOTEBOOK_DIR = pathlib.Path(os.getcwd())  

# root del progetto = la cartella che contiene "src/"
ROOT = NOTEBOOK_DIR.parents[0]  

sys.path.insert(0, str(ROOT))

from src.Heston_FFT import heston_price_fft
from src.Heston_MC import heston_mc_qe_price
from src.Heston_PINN import (
    HestonPINN, scale_inputs, heston_pde_loss, heston_terminal_loss,
    boundary_S0_loss, boundary_Smax_loss, boundary_v_neumann_loss,
    train_heston
)

In [3]:
import time, numpy as np, torch

# HESTON PARAMS 
S0   = 105.0
K    = 105.0
T    = 1.0
r    = 0.05
q    = 0.0

v0     = 0.04
kappa  = 1.5
theta  = 0.04
sigma_v= 0.3
rho    = -0.7

# FFT PARAMS
alpha  = 1.5
fft_N  = 4096      # potenze di 2
eta    = 0.25      # se vuoi piu' finezza sugli strike, riduci (0.05–0.15)

# MC PARAMS
mc_steps = 200
mc_paths = 200_000
seed     = 42

In [4]:
t0 = time.perf_counter()
price_fft = heston_price_fft(S0, K, T, r, v0, kappa, theta, sigma_v, rho,
                             option_type="call", q=q, alpha=alpha, N=fft_N, eta=eta)
t1 = time.perf_counter()
dt_fft = t1 - t0
print(f"FFT:  {price_fft:.6f}  ({dt_fft*1e3:.2f} ms)")


t0 = time.perf_counter()
price_mc, stderr_mc = heston_mc_qe_price(S0, K, T, r, q, v0, kappa, theta, sigma_v, rho,
                                         steps=mc_steps, paths=mc_paths,
                                         antithetic=True, seed=seed, option_type="call")
t1 = time.perf_counter()
dt_mc = t1 - t0
print(f"MC  :  {price_mc:.6f}  ± {1.96*stderr_mc:.6f} (95% CI)  ({dt_mc:.2f} s)")

FFT:  10.879962  (34.67 ms)
MC  :  7.405017  ± 0.022117 (95% CI)  (2.75 s)


In [5]:
'''PINN Train'''

# costruisci e allena "rapido" (poi farai run piu' lunghi)
model = HestonPINN()
_ = train_heston(model,
                 epochs=3000, lr=1e-3,
                 K=K, T=T, S_max=4*S0, v_max=0.8,
                 r=r, q=q, rho=rho, kappa=kappa, theta=theta, sigma=sigma_v,
                 batch_pde=4096, seed=0)

# inference PINN a t=0, S=S0, v=v0
def pinn_price_at_spot(model, S, K, T, v0):
    model.eval()
    with torch.no_grad():
        S_t = torch.tensor([S], dtype=torch.float64)
        v_t = torch.tensor([v0], dtype=torch.float64)
        t_t = torch.tensor([0.0], dtype=torch.float64)
        x = scale_inputs(S_t, v_t, t_t, K=K, T=T, v_max=0.8)
        V = model(x).item()
    return V

t0 = time.perf_counter()
price_pinn = pinn_price_at_spot(model, S0, K, T, v0)
t1 = time.perf_counter()
dt_pinn = t1 - t0
print(f"PINN: {price_pinn:.6f}  ({dt_pinn*1e3:.3f} ms)")

[ep 0] total=2.5637e+05 | pde=4.5448e-04 T=1.5540e+04 S0=3.2555e-03 Smax=1.0096e+05
[ep 500] total=1.8939e+05 | pde=8.2573e+01 T=1.1036e+04 S0=5.3863e-01 Smax=7.8944e+04
[ep 1000] total=1.4081e+05 | pde=1.4524e+02 T=7.9002e+03 S0=2.5247e-02 Smax=6.1658e+04
[ep 1500] total=9.5424e+04 | pde=1.3938e+02 T=4.8688e+03 S0=1.2509e-04 Smax=4.6589e+04
[ep 2000] total=6.7108e+04 | pde=2.9538e+02 T=3.3177e+03 S0=3.7786e-03 Smax=3.3629e+04
[ep 2500] total=4.2464e+04 | pde=4.3083e+02 T=1.9033e+03 S0=1.3983e-02 Smax=2.2992e+04
PINN: 12.126140  (4.471 ms)


**RESULTS**

In [13]:
def bps(x): return 1e4 * x  # 1 bp = 1e-4

err_mc_bps   = bps(abs(price_mc   - price_fft) / max(1.0, price_fft))
err_pinn_bps = bps(abs(price_pinn - price_fft) / max(1.0, price_fft))

print(f"\nPrices: FFT={price_fft:.3f}  |  MC={price_mc:.3f}  |  PINN = {price_pinn:.3f}")
print(f"Errors vs FFT (bps): MC={err_mc_bps:.2f}  |  PINN={err_pinn_bps:.2f}")
print(f"Time: FFT={dt_fft*1e3:.2f} ms  |  MC={dt_mc:.2f} s  |  PINN {dt_pinn*1e3:.2f} ms")


Prices: FFT=10.880  |  MC=7.405  |  PINN = 12.126
Errors vs FFT (bps): MC=3193.89  |  PINN=1145.39
Time: FFT=34.67 ms  |  MC=2.75 s  |  PINN 4.47 ms
